In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math

In [2]:
city = 'seattle'

In [3]:
#read in labels
labels = pd.read_csv('../../../data/' + city + '/processed-labels/labels_raw_' + city + '.csv')
labels

,label_id,label_type,severity,gsv_panorama_id,zoom,heading,pitch,photographer_heading,photographer_pitch,user_id,lat,lng,geometry
0,85055,CurbRamp,1.0,NjPAkwTxWsayAq3kCugvdA,1,107.750000,-15.62500,270.556610,0.907036,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.618599,-122.332787,POINT (-122.3327865600586 47.61859893798828)
1,85057,Obstacle,2.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,38.375000,-21.87500,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617805,-122.331886,POINT (-122.3318862915039 47.61780548095703)
2,85059,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,131.187500,-24.50000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617512,-122.331833,POINT (-122.33183288574219 47.61751174926758)
3,85060,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,131.187500,-24.50000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617519,-122.331879,POINT (-122.33187866210938 47.61751937866211)
4,85062,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,257.187500,-35.00000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617638,-122.332047,POINT (-122.33204650878906 47.617637634277344)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195538,231270,CurbRamp,1.0,R-sTl5O6sOdVPurgTWy3EQ,1,332.468750,-35.00000,215.130295,-0.157570,155af0a9-4149-4436-8aa3-7573e160036d,47.540771,-122.267258,POINT (-122.26725769042969 47.540771484375)
195539,231285,NoSidewalk,5.0,N5MrAAtHZO2JGXUrYAYnSQ,3,0.890625,-11.65625,179.161057,-5.825714,155af0a9-4149-4436-8aa3-7573e160036d,47.552292,-122.263443,POINT (-122.26344299316406 47.55229187011719)
195540,231338,CurbRamp,1.0,d3oBrJ-nDtKMwlE7svc6pg,1,253.562500,-35.00000,177.959915,-0.962440,3ac3d4de-a70a-48ec-b46e-498d6e26ee6c,47.563271,-122.281792,POINT (-122.28179168701172 47.563270568847656)
195541,231339,CurbRamp,1.0,AOIBJ12BjFanDJF1Ark05Q,1,110.312500,-35.00000,179.365723,-1.093400,3ac3d4de-a70a-48ec-b46e-498d6e26ee6c,47.563469,-122.281654,POINT (-122.28165435791016 47.56346893310547)


In [5]:
#print column names into list
list(labels.columns)

['label_id',
 'label_type',
 'severity',
 'gsv_panorama_id',
 'zoom',
 'heading',
 'pitch',
 'photographer_heading',
 'photographer_pitch',
 'user_id',
 'lat',
 'lng',
 'geometry']

In [7]:
#read in all files in the feature folder
clustered = pd.read_csv('../../../data/' + city + '/features/clustered.csv')
distance_to_road = pd.read_csv('../../../data/' + city + '/features/distance_to_road.csv')
distance_to_intersection = pd.read_csv('../../../data/' + city + '/features/distance_to_intersection.csv')
optional_input = pd.read_csv('../../../data/' + city + '/features/optional_input.csv')

In [8]:
df = labels.merge(clustered, on='label_id').merge(distance_to_road, on='label_id').merge(distance_to_intersection, on='label_id').merge(optional_input, on='label_id')

In [9]:
#keep only the label types Curb Ramp, Missing Curb Ramp, Obstacle in Path, Surface Problem, and No Sidewalk
df = df[df['label_type'].isin(['CurbRamp', 'NoCurbRamp', 'Obstacle', 'SurfaceProblem', 'NoSidewalk'])]

In [10]:
#drop nan in severity
df = df.dropna(subset=['severity'])

In [11]:
df

,label_id,label_type,severity,gsv_panorama_id,zoom,heading,pitch,photographer_heading,photographer_pitch,user_id,lat,lng,geometry,cluster_id,clustered,cluster_label_count,way_type,distance_to_road,distance_to_intersection,tag,tag_count,description
0,85055,CurbRamp,1.0,NjPAkwTxWsayAq3kCugvdA,1,107.750000,-15.62500,270.556610,0.907036,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.618599,-122.332787,POINT (-122.3327865600586 47.61859893798828),166316,0,1,primary,26.840775,59.130699,0.0,0.0,0.0
1,85057,Obstacle,2.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,38.375000,-21.87500,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617805,-122.331886,POINT (-122.3318862915039 47.61780548095703),191230,0,1,tertiary,49.480803,0.000000,0.0,0.0,1.0
2,85059,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,131.187500,-24.50000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617512,-122.331833,POINT (-122.33183288574219 47.61751174926758),152398,1,2,tertiary,22.804907,37.121175,0.0,0.0,0.0
3,85060,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,131.187500,-24.50000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617519,-122.331879,POINT (-122.33187866210938 47.61751937866211),155432,0,1,primary,19.008081,33.832501,0.0,0.0,0.0
4,85062,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,257.187500,-35.00000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617638,-122.332047,POINT (-122.33204650878906 47.617637634277344),148478,0,1,tertiary,22.626325,46.324623,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195538,231270,CurbRamp,1.0,R-sTl5O6sOdVPurgTWy3EQ,1,332.468750,-35.00000,215.130295,-0.157570,155af0a9-4149-4436-8aa3-7573e160036d,47.540771,-122.267258,POINT (-122.26725769042969 47.540771484375),86474,1,2,residential,19.281464,33.457578,0.0,0.0,0.0
195539,231285,NoSidewalk,5.0,N5MrAAtHZO2JGXUrYAYnSQ,3,0.890625,-11.65625,179.161057,-5.825714,155af0a9-4149-4436-8aa3-7573e160036d,47.552292,-122.263443,POINT (-122.26344299316406 47.55229187011719),120452,1,2,residential,7.199212,0.000000,0.0,0.0,0.0
195540,231338,CurbRamp,1.0,d3oBrJ-nDtKMwlE7svc6pg,1,253.562500,-35.00000,177.959915,-0.962440,3ac3d4de-a70a-48ec-b46e-498d6e26ee6c,47.563271,-122.281792,POINT (-122.28179168701172 47.563270568847656),86471,0,1,residential,14.445776,152.820404,0.0,0.0,0.0
195541,231339,CurbRamp,1.0,AOIBJ12BjFanDJF1Ark05Q,1,110.312500,-35.00000,179.365723,-1.093400,3ac3d4de-a70a-48ec-b46e-498d6e26ee6c,47.563469,-122.281654,POINT (-122.28165435791016 47.56346893310547),86461,0,1,residential,20.171446,225.060263,0.0,0.0,0.0


In [12]:
gt = pd.read_csv('../../../data/' + city + '/features/' + city + '_ground_truth_labels.csv')
gt

,label_id,ground_truth
0,9,1
1,12,1
2,20,1
3,22,1
4,23,1
...,...,...
16575,233166,0
16576,233167,1
16577,233171,1
16578,233172,1


In [13]:
#merge gt with labels
test= pd.merge(df, gt, on='label_id', how='inner')

In [14]:
#rename ground truth to verified
test.rename(columns={'ground_truth': 'verified'}, inplace=True)

In [15]:
#if the label_id in gt take it out of df
training = df[~df.label_id.isin(test.label_id)]

In [16]:
training

,label_id,label_type,severity,gsv_panorama_id,zoom,heading,pitch,photographer_heading,photographer_pitch,user_id,lat,lng,geometry,cluster_id,clustered,cluster_label_count,way_type,distance_to_road,distance_to_intersection,tag,tag_count,description
0,85055,CurbRamp,1.0,NjPAkwTxWsayAq3kCugvdA,1,107.750000,-15.62500,270.556610,0.907036,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.618599,-122.332787,POINT (-122.3327865600586 47.61859893798828),166316,0,1,primary,26.840775,59.130699,0.0,0.0,0.0
1,85057,Obstacle,2.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,38.375000,-21.87500,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617805,-122.331886,POINT (-122.3318862915039 47.61780548095703),191230,0,1,tertiary,49.480803,0.000000,0.0,0.0,1.0
2,85059,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,131.187500,-24.50000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617512,-122.331833,POINT (-122.33183288574219 47.61751174926758),152398,1,2,tertiary,22.804907,37.121175,0.0,0.0,0.0
3,85060,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,131.187500,-24.50000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617519,-122.331879,POINT (-122.33187866210938 47.61751937866211),155432,0,1,primary,19.008081,33.832501,0.0,0.0,0.0
4,85062,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,257.187500,-35.00000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617638,-122.332047,POINT (-122.33204650878906 47.617637634277344),148478,0,1,tertiary,22.626325,46.324623,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195538,231270,CurbRamp,1.0,R-sTl5O6sOdVPurgTWy3EQ,1,332.468750,-35.00000,215.130295,-0.157570,155af0a9-4149-4436-8aa3-7573e160036d,47.540771,-122.267258,POINT (-122.26725769042969 47.540771484375),86474,1,2,residential,19.281464,33.457578,0.0,0.0,0.0
195539,231285,NoSidewalk,5.0,N5MrAAtHZO2JGXUrYAYnSQ,3,0.890625,-11.65625,179.161057,-5.825714,155af0a9-4149-4436-8aa3-7573e160036d,47.552292,-122.263443,POINT (-122.26344299316406 47.55229187011719),120452,1,2,residential,7.199212,0.000000,0.0,0.0,0.0
195540,231338,CurbRamp,1.0,d3oBrJ-nDtKMwlE7svc6pg,1,253.562500,-35.00000,177.959915,-0.962440,3ac3d4de-a70a-48ec-b46e-498d6e26ee6c,47.563271,-122.281792,POINT (-122.28179168701172 47.563270568847656),86471,0,1,residential,14.445776,152.820404,0.0,0.0,0.0
195541,231339,CurbRamp,1.0,AOIBJ12BjFanDJF1Ark05Q,1,110.312500,-35.00000,179.365723,-1.093400,3ac3d4de-a70a-48ec-b46e-498d6e26ee6c,47.563469,-122.281654,POINT (-122.28165435791016 47.56346893310547),86461,0,1,residential,20.171446,225.060263,0.0,0.0,0.0


In [17]:
training.to_csv('../../../data/' + city + '/processed-labels/training_set_' + city + '.csv', index=False)
test.to_csv('../../../data/' + city + '/processed-labels/test_set_' + city + '.csv', index=False)
df.to_csv('../../../data/' + city + '/processed-labels/labels_full_features_' + city + '.csv', index=False)